In [57]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

In [58]:
threads_link = []
links = []
results = []
threads = []

In [59]:
def scrape_links():
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = "https://jogja.tribunnews.com/diy"
    driver = webdriver.Chrome()
    driver.get(url)
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # Tunggu beberapa detik agar halaman dimuat
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        last_height = new_height
        
    page_source = driver.page_source
    driver.quit()
    
    # Menghapus baris ini karena tidak perlu melakukan permintaan HTTP ke page_source
    # response = requests.get(page_source)
    
    soup = BeautifulSoup(page_source, "html.parser")
    articles = soup.find_all('h3')
    
    page_links = []
    for article in articles:
        link_element = article.find('a')
        if link_element is not None:
            link = link_element.get('href')
            if link is not None:
                page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {url}")
    return page_links

links = scrape_links()
print("Total Links:", len(links))


Scraped 505 links from page https://jogja.tribunnews.com/diy
Total Links: 505


In [60]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Judul Berita
            title_elem = soup.find('h1', {"id": "arttitle"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # # Author berita
            author_elem = soup.find('h5', {"id": "penulis"})
            if author_elem:
                penulis = author_elem.find('b')
                author_text =penulis.text
            else:
                author_text = "Author not found"
            #    # Editor Berita
            # editor_elem = soup.find('div', {"id": "editor"})
            
            # if editor_elem:
            #     editors = editor_elem.find('a')
            #     editor_text = editors.text
            # else:
            #     editor_text = "editor not found" 
            
            # # tanggal berita
            date_elem = soup.find('div', {"class": "grey bdr3 pb10 pt10"})
            if date_elem:
                date_text = date_elem.text.strip()
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find_all('span',{"itemprop": "name"})
            # if category_elements:
            #     category_text= category_elements[2].text
            # else:
            category_text = "Category not found"
            # ## Content Berita
            body_elem = soup.find('div', {"class": "side-article txt-article multi-fontsize editcontent"})
            
            if body_elem:
                content_elem = body_elem.find_all('p')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
              content_text ="Content not found"

            results.append({'title': title_text,
                            'author': author_text,
                            'date':date_text,
                            'category': category_text,
                            'content': content_text,
                            'region':'jogja',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [61]:

for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

In [62]:
df = pd.DataFrame(results)
print("Total Links:", len(results))
df.head(10)

Total Links: 505


,title,author,date,category,content,link
0,Perempuan Rawan Sosial Ekonomi di Sleman Dapat...,Ahmad Syarifudin,"Selasa, 26 September 2023 17:46 WIB",Category not found,"TRIBUNJOGJA.COM, SLEMAN - Pemerintah Kabupaten...",https://jogja.tribunnews.com/2023/09/26/peremp...
1,Pelajari Kebermanfaatan Bambu Lewat ToT Bamboo...,Sri Cahyani Putri,"Selasa, 26 September 2023 20:26 WIB",Category not found,"Laporan Reporter Tribun Jogja, Sri Cahyani Put...",https://jogja.tribunnews.com/2023/09/26/pelaja...
2,Pekan Kompetisi Madrasah Digelar di Kulon Prog...,Alexander Aprita,"Rabu, 27 September 2023 15:46 WIB",Category not found,"TRIBUNJOGJA.COM, KULON PROGO - Kantor Kementer...",https://jogja.tribunnews.com/2023/09/27/pekan-...
3,98 Orang Meninggal Dunia Usai Terlibat Laka La...,Neti Istimewa Rukmana,"Selasa, 26 September 2023 17:05 WIB",Category not found,"TRIBUNJOGJA.COM, BANTUL - Polres Bantul mencat...",https://jogja.tribunnews.com/2023/09/26/98-ora...
4,INFO Jadwal dan Lokasi Pemadaman Listrik Rabu ...,Alifia Nuralita Rezqiana,"Selasa, 26 September 2023 17:33 WIB",Category not found,TRIBUNJOGJA.COM - Perusahaan Listrik Negara (P...,https://jogja.tribunnews.com/2023/09/26/info-j...
5,Pelaku Curanmor di Kokap Kulon Progo Diringkus...,Alexander Aprita,"Rabu, 27 September 2023 11:54 WIB",Category not found,"TRIBUNJOGJA.COM, KULON PROGO - Pelaku pencuria...",https://jogja.tribunnews.com/2023/09/27/pelaku...
6,Pemkab Bantul Gelontorkan Anggaran Lebih dari ...,Neti Istimewa Rukmana,"Rabu, 27 September 2023 15:02 WIB",Category not found,"TRIBUNJOGJA.COM, BANTUL - Pemerintah Kabupaten...",https://jogja.tribunnews.com/2023/09/27/pemkab...
7,5 Fakta Perempuan Muda Meninggal di Kamar Kos ...,Bunga Kartikasari,"Selasa, 26 September 2023 18:46 WIB",Category not found,TRIBUNJOGJA.COM - Kabar seorang perempuan muda...,https://jogja.tribunnews.com/2023/09/26/5-fakt...
8,Simak Pendaftaran PPPK Pemda DIY beserta Link ...,Tribun Jogja,"Rabu, 27 September 2023 12:07 WIB",Category not found,TRIBUNJOGJA.COM – Pembukaan PPPK sudah dibuka ...,https://jogja.tribunnews.com/2023/09/27/simak-...
9,Terjadi Perbedaan Pendapat Majelis Hakim Saat ...,Miftahul Huda,"Rabu, 27 September 2023 11:33 WIB",Category not found,"TRIBUNJOGJA.COM, YOGYA - Sidang pembacaan putu...",https://jogja.tribunnews.com/2023/09/27/terjad...
